In [4]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
import glob

# Create the directory structure
def create_dataset_structure(base_dir='brain_tumor_dataset'):
    for split in ['train', 'validation', 'test']:
        for class_name in ['tumor', 'no_tumor']:
            os.makedirs(os.path.join(base_dir, split, class_name), exist_ok=True)
    print(f"Created dataset structure at {base_dir}")

# Organize from two separate class directories
def organize_dataset(source_tumor_dir, source_no_tumor_dir, base_dir='brain_tumor_dataset', 
                     train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    create_dataset_structure(base_dir)

    for class_name, source_dir in [('tumor', source_tumor_dir), ('no_tumor', source_no_tumor_dir)]:
        image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            image_files.extend(glob.glob(os.path.join(source_dir, ext)))
            image_files.extend(glob.glob(os.path.join(source_dir, ext.upper())))

        print(f"Found {len(image_files)} images in {source_dir}")
        random.shuffle(image_files)

        train_end = int(len(image_files) * train_ratio)
        val_end = train_end + int(len(image_files) * val_ratio)

        splits = {
            'train': image_files[:train_end],
            'validation': image_files[train_end:val_end],
            'test': image_files[val_end:]
        }

        for split, files in splits.items():
            target_dir = os.path.join(base_dir, split, class_name)
            for file_path in files:
                shutil.copy2(file_path, os.path.join(target_dir, os.path.basename(file_path)))
            print(f"Copied {len(files)} images to {target_dir}")

# Organize from a single parent directory containing class subdirectories
def organize_from_single_source(source_dir, base_dir='brain_tumor_dataset',
                                train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    create_dataset_structure(base_dir)

    class_dirs = [d for d in os.listdir(source_dir)
                  if os.path.isdir(os.path.join(source_dir, d))]

    for class_name in class_dirs:
        class_dir = os.path.join(source_dir, class_name)

        image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            image_files.extend(glob.glob(os.path.join(class_dir, ext)))
            image_files.extend(glob.glob(os.path.join(class_dir, ext.upper())))

        print(f"Found {len(image_files)} images in class {class_name}")
        random.shuffle(image_files)

        train_end = int(len(image_files) * train_ratio)
        val_end = train_end + int(len(image_files) * val_ratio)

        splits = {
            'train': image_files[:train_end],
            'validation': image_files[train_end:val_end],
            'test': image_files[val_end:]
        }

        for split, files in splits.items():
            target_dir = os.path.join(base_dir, split, class_name)
            os.makedirs(target_dir, exist_ok=True)
            for file_path in files:
                shutil.copy2(file_path, os.path.join(target_dir, os.path.basename(file_path)))
            print(f"Copied {len(files)} images to {target_dir}")

# Example usage
if __name__ == "__main__":
    random.seed(42)

    # Option 1: Separate tumor and no_tumor directories
    # organize_dataset(
    #     source_tumor_dir="/path/to/tumor",
    #     source_no_tumor_dir="/path/to/no_tumor",
    #     base_dir="brain_tumor_dataset"
    # )

    # Option 2: Single directory with subdirectories 'tumor' and 'no_tumor'
    organize_from_single_source(
        source_dir="brain_data",  # should contain subfolders 'tumor' and 'no_tumor'
        base_dir="brain_tumor_dataset"
    )

    print("Brain tumor dataset organization complete.")


Created dataset structure at brain_tumor_dataset
Brain tumor dataset organization complete.


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define paths - update these to your actual data locations
train_data_dir = 'brain_tumor_dataset/train'
validation_data_dir = 'brain_tumor_dataset/validation'

# Image parameters
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)  # RGB images

# Training parameters
batch_size = 32
epochs = 20

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # 'tumor' vs 'no_tumor'
)

# Load validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output
])

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save model
model.save('brain_tumor_detection_model.h5')

# Plot training history
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs_range = range(len(acc))
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.savefig('brain_training_history.png')
    plt.show()

plot_training_history(history)

# Evaluate on validation set
validation_generator.reset()
y_pred = model.predict(validation_generator, steps=validation_generator.samples // batch_size + 1)
y_pred_classes = (y_pred > 0.5).astype(int)
y_true = validation_generator.classes

# Classification report
print(classification_report(y_true, y_pred_classes))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('brain_confusion_matrix.png')
plt.show()

# Function to predict on new brain MRI images
def predict_brain_tumor(image_path):
    from tensorflow.keras.preprocessing import image
    
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    prediction = model.predict(img_array)[0][0]
    result = "Tumor" if prediction > 0.5 else "No Tumor"
    confidence = prediction if prediction > 0.5 else 1 - prediction
    
    print(f"Prediction: {result} (Confidence: {confidence:.2f})")
    
    plt.imshow(img)
    plt.title(f"{result} (Confidence: {confidence:.2f})")
    plt.axis('off')
    plt.show()
    
    return result, confidence

# Example usage:
# predict_brain_tumor('path/to/test/image.jpg')


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


C:\Users\anees\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     9,437,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,679,041 (36.92 MB)

 Trainable params: 9,679,041 (36.92 MB)

 Non-trainable params: 0 (0.00 B)

C:\Users\anees\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Define image dimensions
img_width, img_height = 224, 224
batch_size = 32

# Build the model (or load a pre-trained one)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: tumor or no tumor
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# If you have a pre-trained model, load it instead
# model = tf.keras.models.load_model('brain_tumor_model.h5')

# Function to process webcam frames and detect brain tumor (from image input)
def detect_brain_tumor_webcam():
    cap = cv2.VideoCapture(0)  # Use default webcam

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    print("Webcam opened successfully. Show a brain MRI image. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Error: Failed to capture image")
            break

        # Resize frame for model input
        input_frame = cv2.resize(frame, (img_width, img_height))

        # Preprocess
        input_array = np.expand_dims(input_frame, axis=0) / 255.0

        # Predict
        prediction = model.predict(input_array)[0][0]
        result = "Tumor" if prediction > 0.5 else "No Tumor"
        confidence = prediction if prediction > 0.5 else 1 - prediction

        # Annotate frame
        text = f"{result} (Conf: {confidence:.2f})"
        color = (0, 0, 255) if result == "Tumor" else (0, 255, 0)
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        # Display
        cv2.imshow('Brain Tumor Detection', frame)

        # Quit key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Plot training history
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.savefig('training_history.png')
    plt.show()

# Predict single MRI image
def predict_brain_tumor(image_path):
    from tensorflow.keras.preprocessing import image

    img = image.load_img(image_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    prediction = model.predict(img_array)[0][0]
    result = "Tumor" if prediction > 0.5 else "No Tumor"
    confidence = prediction if prediction > 0.5 else 1 - prediction

    print(f"Prediction: {result} (Confidence: {confidence:.2f})")

    plt.imshow(img)
    plt.title(f"{result} (Confidence: {confidence:.2f})")
    plt.axis('off')
    plt.show()

    return result, confidence

# Start webcam detection
if __name__ == "__main__":
    detect_brain_tumor_webcam()


Webcam opened successfully. Show a brain MRI image. Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
